In [1]:
import os
import math
import tflearn
import pandas as pd
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
import sklearn.linear_model as model
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import sklearn
from sklearn.naive_bayes import BernoulliNB
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout

Using TensorFlow backend.


### Basic Logistic Regression

Use Michelle's method for importing data since I can't do it correctly:

In [2]:
train_data = np.genfromtxt('Data/training_data.txt',dtype='str')
test_data = np.genfromtxt('Data/test_data.txt',dtype='str')
#train_data = np.loadtxt('training_data.txt',  delimiter=' ')
#test_data = np.loadtxt('test_data.txt')
train_labels = train_data[0, :]
train_stars = train_data[1:, 0]
train_reviews = train_data[1:, 1:]

test_labels = test_data[0, :]
#test_stars = train_data[1:, 0]
test_reviews = test_data[1:, 0:]

trainX = train_reviews
trainY = train_stars
testX = test_reviews
#testY = test_stars

print("trainX = ", trainX.shape)
print("trainY = ", trainY.shape)
print("testX = ", testX.shape)


for i in range(0, len(trainX)):
    for j in range(0, len(trainX[0])):
        trainX[i][j] = int(trainX[i][j])

for i in range(0, len(testX)):
    for j in range(0, len(testX[0])):
        testX[i][j] = int(testX[i][j])

for i in range(0, len(trainY)):
        trainY[i] = int(trainY[i])
#for i in range(0, len(trainY)):
        #testY[i] = int(testY[i])

trainX =  (20000, 1000)
trainY =  (20000,)
testX =  (10000, 1000)


Define a function for classification error:

In [3]:
def class_error(y, y_pred):
    misclassified = 0
    for i in range(len(y)):
        if y[i] != y_pred[i]:
            misclassified += 1
    return float(misclassified) / len(y)

Split data into 80% training set and 20% test set.

In [4]:
num_rows = []
for i in range(len(trainY)):
    num_rows.append(i)
    
np.random.shuffle(num_rows)
train_indices = num_rows[:16000]
valid_indices = num_rows[16000:]

train_X = []
train_y = []
valid_X = []
valid_y = []

for index in train_indices:
    train_X.append(trainX[index])
    train_y.append(trainY[index])
    
for index in valid_indices:
    valid_X.append(trainX[index])
    valid_y.append(trainY[index])

With Michelle's code, it doesn't actually convert the array contents into integers because she tried to do it in place. This code does it correctly out of place:

In [5]:
train_X2 = []
for row in train_X:
    temp = []
    for element in row:
        temp.append(int(element))
    train_X2.append(temp)

valid_X2 = []
for row in valid_X:
    temp = []
    for element in row:
        temp.append(int(element))
    valid_X2.append(temp)
    
valid_y2 = []
for row in valid_y:
    valid_y2.append(int(row))
    
train_y2 = []
for row in train_y:
    train_y2.append(int(row))
    
test_X2 = []
for row in testX:
    temp = []
    for element in row:
        temp.append(int(element))
    test_X2.append(temp)

Convert everything back to a numpy array:

In [6]:
trainX = np.array(train_X2)
validX = np.array(valid_X2)
trainY = np.array(train_y2)
validY = np.array(valid_y2)
testX = np.array(test_X2)

# Basic Neural Network 

In [7]:
print(trainY)

[1 1 1 ..., 0 1 1]


In [8]:
print(validY)

[1 0 1 ..., 1 1 0]


In [9]:
trainY = to_categorical(trainY, 2)

In [10]:
print(trainY)

[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 ..., 
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]]


In [11]:
validY = to_categorical(validY, 2)
print(validY)

[[ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 ..., 
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [40]:
model = Sequential()

keras.layers.BatchNormalization(axis=-1, 
momentum=0.99, epsilon=0.001, center=True, 
scale=True, beta_initializer='zeros', 
gamma_initializer='ones', moving_mean_initializer='zeros', 
moving_variance_initializer='ones', beta_regularizer=None, 
gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)


model.add(Dense(60, input_dim=1000))
keras.layers.LeakyReLU(alpha=0.3)
#model.add(Activation('leaky_relu'))
model.add(Dropout(0.20))

model.add(Dense(60)) 
#model.add(Activation('leaky_relu'))
keras.layers.LeakyReLU(alpha=0.3)
model.add(Dropout(0.20))

model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

fit = model.fit(trainX, trainY, batch_size=64, nb_epoch=35, validation_data=(validX, validY),
    verbose=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 60)                60060     
_________________________________________________________________
dropout_15 (Dropout)         (None, 60)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 60)                3660      
_________________________________________________________________
dropout_16 (Dropout)         (None, 60)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 122       
_________________________________________________________________
activation_7 (Activation)    (None, 2)                 0         
Total params: 63,842
Trainable params: 63,842
Non-trainable params: 0
_________________________________________________________________


/Users/michellezhao/anaconda3/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 16000 samples, validate on 4000 samples
Epoch 1/35
16000/16000 [==============================] - 1s 72us/step - loss: 0.4328 - acc: 0.8034 - val_loss: 0.3663 - val_acc: 0.8420
Epoch 2/35
16000/16000 [==============================] - 1s 38us/step - loss: 0.3526 - acc: 0.8520 - val_loss: 0.3622 - val_acc: 0.8440
Epoch 3/35
16000/16000 [==============================] - 1s 37us/step - loss: 0.3376 - acc: 0.8581 - val_loss: 0.3598 - val_acc: 0.8427
Epoch 4/35
16000/16000 [==============================] - 1s 37us/step - loss: 0.3342 - acc: 0.8553 - val_loss: 0.3570 - val_acc: 0.8462
Epoch 5/35
16000/16000 [==============================] - 1s 39us/step - loss: 0.3299 - acc: 0.8620 - val_loss: 0.3596 - val_acc: 0.8455
Epoch 6/35
16000/16000 [==============================] - 1s 38us/step - loss: 0.3286 - acc: 0.8617 - val_loss: 0.3587 - val_acc: 0.8440
Epoch 7/35
16000/16000 [==============================] - 1s 38us/step - loss: 0.3236 - acc: 0.8656 - val_loss: 0.3620 - val_acc:

In [41]:
print(trainY)

[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 ..., 
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]]


Fit the model:

Print the classification error on the training set:

In [42]:
prediction = model.predict(trainX)
predictedY = []

trainY_new = []
for i in range(0, len(trainY)):
    if trainY[i][0] == 1:
        trainY_new.append(0)
    else:
        trainY_new.append(1)



for i in range(0, len(prediction)):
    if abs(prediction[i][0]-1) < abs(prediction[i][1]-1):
        predictedY.append(0)
    else:
        predictedY.append(1)
        
class_error(predictedY, trainY_new)

0.1230625

In [43]:
prediction_V = model.predict(validX)
predictedY_V= []

validY_new = []
for i in range(0, len(validY)):
    if validY[i][0] == 1:
        validY_new.append(0)
    else:
        validY_new.append(1)



for i in range(0, len(prediction_V)):
    if abs(prediction_V[i][0]-1) < abs(prediction_V[i][1]-1):
        predictedY_V.append(0)
    else:
        predictedY_V.append(1)
        
class_error(predictedY_V, validY_new)

0.1605

In [39]:
test_predictions = model.predict(testX)
predictions_arr = []

for i in range(0, len(test_predictions)):
    if abs(test_predictions[i][0]-1) < abs(test_predictions[i][1]-1):
        predictions_arr.append(0)
    else:
        predictions_arr.append(1)
    
writeToText(predictions_arr, 'basicNeuralNet2.txt')

Print the classification error on the validation set:

A function for writing to the text file for the submission:

In [38]:
def writeToText(predictions, filename):
    array = [["Id","Prediction"]]
    index = 1
    for i in range(len(predictions)):
        predict = predictions[index - 1]
        array.append([index, int(predictions[index - 1])])
        index += 1
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter=',', quotechar='|')
    writer.writerows(array)

### Logistic Regression with TF-IDF

A function to convert the set into a TF-IDF matrix:

In [ ]:
def TFIDF(X, labels):
    documents = []

    for element in X:
        temp = ''
        for i in range(len(element)):
            word = labels[i + 1] + ' '
            new_word = word * element[i]
            temp += (new_word)
        documents.append(temp)
        
    tokenize = lambda doc: doc.lower().split(" ")
    sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
    return sklearn_tfidf.fit_transform(documents)

Fit the model using the training data:

In [ ]:
model = Sequential()
model.add(Dense(1000, input_dim=1000))
model.add(Activation('relu'))
model.add(Dropout(0.20))

model.add(Dense(1000)) 
model.add(Activation('relu'))
model.add(Dropout(0.20))

model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

inputX = TFIDF(train_X2, train_labels)
print(inputX)
print(len(inputX))

fit = model.fit(inputX, trainY, batch_size=64, nb_epoch=15, validation_data=(validX, validY),
    verbose=1)


prediction = model.predict(trainX)
predictedY = []

trainY_new = []
for i in range(0, len(trainY)):
    if trainY[i][0] == 1:
        trainY_new.append(0)
    else:
        trainY_new.append(1)



for i in range(0, len(prediction)):
    if abs(prediction[i][0]-1) < abs(prediction[i][1]-1):
        predictedY.append(0)
    else:
        predictedY.append(1)
        
class_error(predictedY, trainY_new)